JupyterLab Notebook Examples:  
Verbose REFERENTIAL material to be utilized in a LINUX environment.  

GOTCHA - Be mindful of the Default Working Directory setting.  

Install required Python components:  

        sudo dnf install pip  

        pip install et_xmlfile --user  
        pip install openpyl --user  
        pip install sqlalchemy --user  
        pip install xlswriter --user  

Nonexistent directory './data' will be created in  
the Default Working Directory.  

Nonexistent file 'carsales.csv' will be written in './data'.  
Nonexistent file 'carsales.xlsx' will be written in './data'.  

Database file(s) will be written in './data/rdbms'.

In [1]:
'''
'CSV_2_SQLdbTable_2_Dframe.ipynb'      

if CSV file './data/carsales.csv' exists:
    jump to 'HouseKeepingEnded'
    
Create DataFrame 'dic_cardata' from a dictionary
Rename the DataFrame Index.
Write file './data/carsales.csv'.

            HouseKeepingEnded:

Create DataFrame 'df_data' by reading file './data/carsales.csv'.

Write data from DataFrame 'df_data' to
    table 'tblDic1' in database 'TestDic1.db'.
    Do not include the index when writing to the table.
    
Write data from DataFrame 'df_data' to
   table 'tblDic2' in database 'TestDic2.db'.
   Include the index when writing to the table.

Create DataFrame ' df_tblDic1' by
  reading SQL table 'tblDic1'.

Create DataFrame 'df_tblDic2' by
  reading SQL table 'tblDic2' .

Compare the original DataFrame 'df_data' with a DataFrame
  derived from SQL 'tblDic1'.

Compare the original DataFrame 'df_data' with a DataFrame
  derived from SQL 'tblDic2'.L

Delete all created DataFrames from memory.
'''
#
from IPython.display import display
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR
import os
import pandas as pd
import time
#

# Ensure that directory './data' exists and can be written to.
# Ensure that directory './data/rdbms' exists and can be written to.
# Save the DataFrame as a CSV file while
#   including the Index and Index Labels (default).
# Update './data' datetime stamp
name_dir = './data'
name_dir_db = 'data/rdbms'
os_dir_db = './' +  name_dir_db
os.makedirs(name_dir, exist_ok=True)
os.chmod(name_dir, 0o755)
os.makedirs(os_dir_db, exist_ok=True)
os.chmod(os_dir_db, 0o755)

# Create an empty DataFrame to facilitate deletion WITHOUT an error
#   if CSV file './data/carsales.csv' exists and
#   DataFrame df_carsales' was not populated from a dynamically
#   created CSV file.
#
# If CSV file './data/carsales.csv' was not found, create it then
#   rename the Index of DataFrame 'df_carsales'.
f_created_csv = False
name_filepath = './data/carsales.csv'
if os.path.exists(name_filepath):
    os.chmod(name_filepath, 0o444)
    df_carsales = pd.DataFrame()
else:
    f_created_csv = True

    dic_cardata = {
        'Hyundai': {
            'One': 2, 'Two': 6, 'Three': 2, 'Four': 3, 'Five': 0,
            'Six': 4, 'Seven': 7,
        },
        'Mazda': {
            'One': 12, 'Two': 11, 'Three': 0, 'Four': 0, 'Five': 0,
            'Six': 4, 'Seven': 1,
        },
        'Mercedes': {
            'One': 2, 'Two': 4, 'Three': 0, 'Four': 4, 'Five': 0,
            'Six': 3, 'Seven': 3,
        },
        'Ford': {
            'One': 3, 'Two': 0, 'Three': 0, 'Four': 1, 'Five': 6,
            'Six': 12, 'Seven': 5,
        },
        'BYD': {
            'One': 9, 'Two': 3, 'Three': 4, 'Four': 1, 'Five': 0,
            'Six': 0, 'Seven': 1,
        },
        'Renault': {
            'One': 12, 'Two': 1, 'Three': 0, 'Four': 0, 'Five': 3,
            'Six': 1, 'Seven': 1,
        },
        'Volvo': {
            'One': 1, 'Two': 4, 'Three': 0, 'Four': 0, 'Five': 0,
            'Six': 11, 'Seven': 12,
        },
    }
    df_carsales = pd.DataFrame(dic_cardata)
    df_carsales.index.rename(
        'Sales Location', inplace=True
    )

    # Save the DataFrame as a CSV file while
    #   including the Index and Index Labels (default).
    # Update './data' datetime stamp
                # os_dir_db = './' +  name_dir_db
                # os.makedirs(name_dir, exist_ok=True)
                # os.chmod(name_dir, 0o755)
                # os.makedirs(os_dir_db, exist_ok=True)
                # os.chmod(os_dir_db, 0o755)
    with open(name_filepath, 'w') as writer:
        df_carsales.to_csv(writer)
        os.chmod(name_filepath, 0o444)
    # Get then display modified time info
    ti_m = os.path.getmtime(name_filepath)
    m_ti = time.ctime(ti_m)
    Path(name_dir).touch()
    print(f"\nFile '{name_filepath}' was CREATED: {m_ti}")
        
            # HouseKeepingEnded:

# Create DataFrame 'df_data' by reading './data/carsales.csv'.
# Get the Index via parameter'index_col=0'.
# Store the Index in 'df_data_index'.
df_data = pd.read_csv(name_filepath, index_col=0)
if f_created_csv:
    print('As a dynamically generated CSV file')
print('df_data')
display(df_data)
df_data_index = df_data.index.name

# Create a SQLite database connector.
# Create SQLite database table.
db_name1 = 'TestDic1.db'
db_name2 = 'TestDic2.db'
# Required for connections
db_file1 = name_dir_db + '/' + db_name1
db_file2 = name_dir_db + '/' + db_name2
# Only required to validate existence of filepaths.
db_filepath1 = './' + db_file1
db_filepath2 = './' + db_file2
table_name1 = 'tblDic1'
table_name2 = 'tblDic2'
engine_cmd1 = 'sqlite:///' + db_file1
engine_cmd2 = 'sqlite:///' + db_file2
    
# print(f'\nengine_cmd1:\n{engine_cmd1}')
# print(f'\nengine_cmd2:\n{engine_cmd2}')

if os.path.exists(db_filepath1):
    os.chmod(db_filepath1, 0o644)
if os.path.exists(db_filepath2):
    os.chmod(db_filepath2, 0o644)

# Create SQLite database connectors and connection strings.
engine1 = create_engine(engine_cmd1)
engine2 = create_engine(engine_cmd2)
sqlite_connection1 = engine1.connect()
sqlite_connection2 = engine2.connect()

# Write data from DataFrame 'df_data' to
#    table {table_name1} in database {db_name1}.
# Remove Index column, replacing it with default incrementing digits.
df_data.to_sql(
    name=table_name1, con=sqlite_connection1,
    if_exists='replace', index=False
)

# Write data from DataFrame 'df_data' to
#    table {table_name2} in database {db_name2}.
# Include Index column.
df_data.to_sql(
    name=table_name2, con=sqlite_connection2,
    if_exists='replace', index=True
)
sqlite_connection1.close()
sqlite_connection2.close()
if os.path.exists(db_filepath1):
    os.chmod(db_filepath1, 0o644)
if os.path.exists(db_filepath2):
    os.chmod(db_filepath2, 0o644)

# Create DataFrame ' df_tblDic1' by
#   reading SQL table {table_name1}.
df_tblDic1 = pd.read_sql_table(
    table_name=table_name1, con=engine1
)
sqlite_connection1.close()
engine1.dispose()
print(
    f'\nDataFrame df_tblDic1:\n'
    f'Formed from a SQL table created VIA:\n' 
    f'"df_data.to_sql(name=table_name1, con=sqlite_connection1,\n'
    f"\tif_exists='replace', index == False\""
)
display(df_tblDic1)

# Create DataFrame ' df_tblDic2' by
#   reading SQL table {table_name2}.
df_tblDic2 = pd.read_sql_table(table_name=table_name2, con=engine2)
sqlite_connection2.close()
engine2.dispose()
print(
    f'\nDataFrame df_tblDic2:\n'
    f'Formed from a SQL table created VIA:\n' 
    f'"df_data.to_sql(name=table_name2, con=sqlite_connection2,\n'
    f"\tif_exists='replace', index == True\""
)
display(df_tblDic2)

# Set the Index of DataFrame 'df_tblDic2'.
df_tblDic2.set_index(
    'Sales Location', inplace=True
)
print(f'df_tblDic2 - pd.read_sql_table set_index')
display(df_tblDic2)

# Compare the original DataFrame 'df_data' with a DataFrame
#   derived from SQL 'tblDic1'.
fequal1 = df_data.equals(df_tblDic1)
print("DataFrame 'df_tblDic1' does NOT contain an Index")
print(f"Comparison 'df_data' vs 'df_tblDic1': {fequal1}")

# Compare the original DataFrame 'df_data' with a DataFrame
#   derived from SQL 'tblDic2'.
fequal2 = df_data.equals(df_tblDic2)
print("\nDataFrame 'df_tblDic1' SHOULD contain an Index")
print(f"Comparison 'df_data' vs 'df_tblDic2': {fequal2}")

# Delete DataFrames 'df_carsales'and 'df_data' from memory.
del df_carsales
del df_data

# Get time info.
time_local = time.localtime()
time_string = time.strftime('%Y-%m-%d %H:%M:%S %Z %z', time_local)
print(f'\n{time_string}')
# EOF


File './data/carsales.csv' was CREATED: Wed Oct  2 18:29:33 2024
As a dynamically generated CSV file
df_data


,Hyundai,Mazda,Mercedes,Ford,BYD,Renault,Volvo
Sales Location,,,,,,,
One,2,12,2,3,9,12,1
Two,6,11,4,0,3,1,4
Three,2,0,0,0,4,0,0
Four,3,0,4,1,1,0,0
Five,0,0,0,6,0,3,0
Six,4,4,3,12,0,1,11
Seven,7,1,3,5,1,1,12



DataFrame df_tblDic1:
Formed from a SQL table created VIA:
"df_data.to_sql(name=table_name1, con=sqlite_connection1,
	if_exists='replace', index == False"


,Hyundai,Mazda,Mercedes,Ford,BYD,Renault,Volvo
0,2,12,2,3,9,12,1
1,6,11,4,0,3,1,4
2,2,0,0,0,4,0,0
3,3,0,4,1,1,0,0
4,0,0,0,6,0,3,0
5,4,4,3,12,0,1,11
6,7,1,3,5,1,1,12



DataFrame df_tblDic2:
Formed from a SQL table created VIA:
"df_data.to_sql(name=table_name2, con=sqlite_connection2,
	if_exists='replace', index == True"


,Sales Location,Hyundai,Mazda,Mercedes,Ford,BYD,Renault,Volvo
0,One,2,12,2,3,9,12,1
1,Two,6,11,4,0,3,1,4
2,Three,2,0,0,0,4,0,0
3,Four,3,0,4,1,1,0,0
4,Five,0,0,0,6,0,3,0
5,Six,4,4,3,12,0,1,11
6,Seven,7,1,3,5,1,1,12


df_tblDic2 - pd.read_sql_table set_index


,Hyundai,Mazda,Mercedes,Ford,BYD,Renault,Volvo
Sales Location,,,,,,,
One,2,12,2,3,9,12,1
Two,6,11,4,0,3,1,4
Three,2,0,0,0,4,0,0
Four,3,0,4,1,1,0,0
Five,0,0,0,6,0,3,0
Six,4,4,3,12,0,1,11
Seven,7,1,3,5,1,1,12


DataFrame 'df_tblDic1' does NOT contain an Index
Comparison 'df_data' vs 'df_tblDic1': False

DataFrame 'df_tblDic1' SHOULD contain an Index
Comparison 'df_data' vs 'df_tblDic2': True

2024-10-02 18:29:33 PDT -0700
